# Configure a GBDX Workflow that Launches a Juno Kernel

This notebook guides you through launching a Jupyter kernel on [GBDX](https://gbdx.geobigdata.io/login), accessible via [Juno](https://juno.timbr.io).  To get started, you will need GBDX Credentials (available from the signup page), a DigitalGlobe Catalog ID for some imagery that you want to work with, and a Juno External Kernel token.


## Set up GBDX and Juno Credentials

Replace the credentials section below with yours. _Instructions on doing that?_


In [1]:
import requests
from gbdxtools import Interface
creds = {
    "client_id": "CLIENT_ID",
    "client_secret": "CLIENT_SECRET",
    "username": "my@email.com",
    "password": "password"
}

JUNO_KERNEL_TOKEN = "JUNO_KERNEL_TOKEN"

gbdx = Interface(**creds)
gbdx_headers = {"Authorization": "Bearer {}".format(gbdx.gbdx_connection.token.get("access_token")), "Content-Type": "application/json"}

## Specify Catalog IDs

One way to find Catalog IDs is to use the [GBDX web application](https://gbdx.geobigdata.io).  These are from around Mexicali (Baja California, Mexico).

In [2]:
catalog_ids = ["105001000126EF00", "101001000BAFE000"]

## Order Imagery

This should complete nearly instantaneously since all the imagery is available

In [3]:
order_id = gbdx.ordering.order(catalog_ids)
data_path = gbdx.ordering.status(order_id)[0]['location']

## Launch The Kernel

We construct a GBDX workflow to do some preprocessing and then provide the resulting imagery to a Juno Kernel that we'll be able to connect to.  This task uses DigitalGlobe algorithms on GBDX to orthorectify and compensate for atmospheric effects.  The results are 8-band multispectral images.  Since this is a computationally intensive operation, it can take some time to complete.  The status of the workflow can be seen using the `workflow.events` call in the 2nd cell below.

Once the Juno kernel is running, its status indicator on the Juno home page will start blinking green.  At this point you can connect to it and run code against the imagery that you have selected.

In [4]:
aoptask = gbdx.Task("AOP_Strip_Processor", data=data_path, bands="MS", enable_acomp=True, enable_pansharpen=False)
junotask = gbdx.Task("Timbr-JunoBaseKernel", JUNO_TOKEN=JUNO_KERNEL_TOKEN)
junotask.inputs.data = aoptask.outputs.data.value
workflow = gbdx.Workflow([aoptask, junotask])
w_id = workflow.execute()

In [6]:
workflow.events

[{u'event': u'submitted',
  u'note': u'',
  u'state': u'pending',
  u'task': u'AOP_Strip_Processor_35116276-ef50-4b92-b22f-6bcc2c905632',
  u'timestamp': u'2016-09-20T02:42:42.427344+00:00',
  u'when': u'11 minutes ago'},
 {u'event': u'submitted',
  u'note': u'',
  u'state': u'pending',
  u'task': u'Timbr-JunoBaseKernel_43e37cf5-dc2a-416e-b09c-4d3fa14f61b7',
  u'timestamp': u'2016-09-20T02:42:42.432381+00:00',
  u'when': u'11 minutes ago'}]

## Find Running GBDX Tasks

The Juno Kernel task will run for 2 days and then time out, we should probably clean up if we don't need it for that long.

In [14]:
query = """
{
    "state": "running"
}
"""
resp = requests.post("https://geobigdata.io/workflows/v1/workflows/search", headers=gbdx_headers, data=query)
workflows = resp.json()
workflows

{u'Workflows': [u'4425754928451478059']}

In [15]:
gbdx.workflow.cancel(workflows["Workflows"][0])